In [0]:
dbutils.widgets.text("proc_date", "", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE  {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sft 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) 
AS
WITH dat AS (
    SELECT  ROW_NUMBER() OVER (PARTITION BY wh_site_id, sku, calday, type ORDER BY proc_date DESC) rn,
            wh_site_id dc_id,
            sku product_id,
            DATE(calday) calday,
            rkt_available_qty,
            sft_po_processing_qty,
            sft_lost_qty,
            sft_physical_lost_qty,
            sft_available_qty,
            sft_damage_qty,
            sft_out_of_date_qty,
            sft_nearing_expiration_date_qty,
            sft_return_to_vendor_qty,
            sft_pending_in_qty,
            sft_not_sync_sap_pending_in_qty,
            rkt_sto_instock_qty,
            sft_after_packing_qty,
            sft_instock_qty,
            sft_pickable_qty,
            IFNULL(sft_pending_in_qty, 0) - IFNULL(sft_not_sync_sap_pending_in_qty, 0) - IFNULL(sft_po_processing_qty, 0) open_order
FROM    {catalog_name}.udp_wcm_silver_realtime.sft_stock
WHERE   client_code = 'WIN' AND type = 'OPENING_STOCK'
)
SELECT * EXCEPT(rn) FROM dat WHERE rn=1;
""")